In [1]:
import sys
sys.path.insert(0, '/usr/src/mytonctrl')

In [2]:
import requests as R
import mytonctrl as M


from datetime import datetime

# Prod

In [42]:
from pymongo import MongoClient

prod_client = MongoClient(host='51.250.2.106', port=27017)
prod_client.list_database_names()

['admin', 'config', 'local', 'production_ton_monitor']

In [44]:
db = prod_client['production_ton_monitor']
db

Database(MongoClient(host=['51.250.2.106:27017'], document_class=dict, tz_aware=False, connect=True), 'production_ton_monitor')

In [45]:
db.list_collection_names()

['transaction_details',
 'ton_bridges',
 'validators',
 'governance',
 'ton_apis',
 'transactions',
 'blocks',
 'liteservers']

In [118]:
result = ton.validatorConsole.Run("getstats").split('\n')
result

['connecting to [127.0.0.1:4351]',
 'local key: 443652FA14CE7BA2A0280622E0D824718422F6C2E00B107D254AAFD0342F85AD',
 'remote key: 91B063AB47BC55B82A8DE69D36D36231A489527CA21802217888BE231C1CE661',
 'conn ready',
 'unixtime\t\t\t1637127512',
 'masterchainblock\t\t\t(-1,8000000000000000,16143168):64EC0D19AD417F37C1931D1124D8964C8CC0F92CBDD025EA263C16D49EC6C660:7EE6E01C268C93371A539C5CD85D247CF65B15C47FB4541E6A43378284B922FA',
 'masterchainblocktime\t\t\t1637127501',
 'gcmasterchainblock\t\t\t(-1,8000000000000000,16067661):1095E800069130A89F743A8FF5003BEEDB6833FE072C2EE9485B418682BBA9EE:653635AD1C19166766F39A8F41A1B5A32BFBC92AA16ADBF8E3F4AA2A15E3CC74',
 'keymasterchainblock\t\t\t(-1,8000000000000000,16141109):BF87EF052C392C0F56C45BB62640AA63946D5E28A4BB38A24FE1C82889639522:4D52989C86EBBC480C879BD137471609AACD617A09013D2F8EF10FD4AE85FA7D',
 'knownkeymasterchainblock\t\t\t(-1,8000000000000000,16141109):BF87EF052C392C0F56C45BB62640AA63946D5E28A4BB38A24FE1C82889639522:4D52989C86EBBC480C879BD13

# reset database

In [ ]:
db_test = get_database('test_db')
x = db_test.test_collection.insert_one({'test': 2})
x.inserted_id

In [ ]:
from bson import ObjectId

In [ ]:
db_test.test_collection.replace_one(Obj)

In [ ]:
db_test.test_collection.update_one({'_id': x.inserted_id}, {"$inc": {"test": -1}})
list(db_test.test_collection.find({}))

In [ ]:
from pymongo import ObjectId
id1 = ObjectId('6194466751b12abd8a94bbd7')

In [ ]:
db_test.test_collection.replace_one({'test': 2}, {"test1": 3, "test2": 4})

In [ ]:
db_test.test_collection.update_one({'_id': x.inserted_id}, {"$inc": {"test": -1}})
list(db_test.test_collection.find({}))

In [ ]:
last

In [ ]:
l = [1, 2, 3, 4, 5, 6]
l.pop(0)
l

In [ ]:
from 

In [ ]:
!pip3 install bencode.py

In [ ]:
{
  "@type":"dht.node",
  "id":{
     "@type":"pub.ed25519",
     "key":"C1uy64rfGxp10SPSqbsxWhbumy5SM0YbvljCudwpZeI="
  },
  "addr_list":{
     "@type":"adnl.addressList",
     "addrs":[
        {
           "@type":"adnl.address.udp",
           "ip":-1307380867,
           "port":15888
        }
     ],
     "version":0,
     "reinit_date":0,
     "priority":0,
     "expire_at":0
  },
  "version":-1,
  "signature":"s+tnHMTzPYG8abau+1dUs8tBJ+CDt+jIPmGfaVd7nmfb1gt6lL10G2IwkNeWhkxjZcAHRc0azWFVxp+IjIOOBQ=="
},

In [25]:
f"{int2ip(-1307380867)}:{15888}"

'178.18.243.125:15888'

In [13]:
hex(15888)

'0x3e10'

In [20]:
old_servers = [
    {'addr': 1, 'val': 10},
    {'addr': 2, 'val': 20},
    {'addr': 3, 'val': 30},
]
new_servers = [
    {'addr': 1, 'val': 10, 'val2': 100},
    {'addr': 4, 'val2': 400},
]

In [22]:
from backend.data.mongodb import get_mongo_client

get_mongo_client().test_db.drop_collection('test2')
liteservers_collection = get_mongo_client().test_db.test2
liteservers_collection.insert_many(old_servers)
list(liteservers_collection.find({}))

[{'_id': ObjectId('6194698d9860638455ea482e'), 'addr': 1, 'val': 10},
 {'_id': ObjectId('6194698d9860638455ea482f'), 'addr': 2, 'val': 20},
 {'_id': ObjectId('6194698d9860638455ea4830'), 'addr': 3, 'val': 30}]

In [ ]:
def update_mongo_collection(new, old, collection, key):
    new_set = {x['key'] for x in new}
    old_set = {x['key'] for x in old}

    to_remove = old_set - new_set
    to_update = old_set & new_set
    to_add = new_set - old_set

    # remove
    for k in to_remove:
        collection.delete_one({key: k})

    # add and update
    for x in new:
        if x[key] in to_update:
            collection.replace_one({key: x[key]}, x)
        else:
            collection.insert_one(x)
    return

In [27]:
ton = M.MyTonCore()

In [35]:
block = {
  "workchain": -1,
  "shard": -9223372036854775808,
  "seqno": 0,
  "root_hash": "F6OpKZKqvqeFp6CQmFomXNMfMj2EnaUSOXN+Mh+wVWk=",
  "file_hash": "XplPz01CXAps5qeSWUtxcyBfdAo5zVb1N979KLSKD24="
}
block = "(-1,8000000000000000,0):17A3A92992AABEA785A7A090985A265CD31F323D849DA51239737E321FB05569:5E994FCF4D425C0A6CE6A792594B7173205F740A39CD56F537DEFD28B48A0F6E"
block = "(-1,8000000000000000,16140693):23291E846C065C9A0CD7C7FC67F5F64BBB39447415336D24FF5A5DA0965D6D23:AF67E40BB6A6FA0E625C3ED6E9B52638A9BF81A016FC3569E53F44350FFC75C6"
block = "(-1,-9223372036854775808,0):F6OpKZKqvqeFp6CQmFomXNMfMj2EnaUSOXN+Mh+wVWk=:XplPz01CXAps5qeSWUtxcyBfdAo5zVb1N979KLSKD24="
ton.liteClient.Run(f"dumpblock {block}")

'latest masterchain block known to server is (-1,8000000000000000,16140715):13FAE27B768FC5F1EAFC2181109AE8835DD642F1709B229ECED69774B7F95081:4361306A6420838016DA3E3DC47F3CC318F54A5734E21DDFAF56B057BD16812A created at 1637118215 (6 seconds ago)\nBLK#1 = (-1,8000000000000000,16140715):13FAE27B768FC5F1EAFC2181109AE8835DD642F1709B229ECED69774B7F95081:4361306A6420838016DA3E3DC47F3CC318F54A5734E21DDFAF56B057BD16812A\nBLK#2 = (-1,8000000000000000,0):17A3A92992AABEA785A7A090985A265CD31F323D849DA51239737E321FB05569:5E994FCF4D425C0A6CE6A792594B7173205F740A39CD56F537DEFD28B48A0F6E\n                                                                                                                               ^\n'

In [ ]:
list(liteservers_collection)

In [18]:
hash('C1uy64rfGxp10SPSqbsxWhbumy5SM0YbvljCudwpZeI=')

574297323898973918

In [ ]:
b212f37d3e10

In [16]:
list(map(hex, map(int, int2ip(-1307380867).split('.'))))

['0xb2', '0x12', '0xf3', '0x7d']

In [ ]:
def get_parent(block):
    block_full = ton.liteClient.Run(f'dumpblock {block}').split('\n')
    prev = None
    for row in block_full()

In [ ]:
def get_last_block():
    result = ton.liteClient.Run("last")
    lines = result.split('\n')
    for line in lines:
        if "latest masterchain block" in line:
            buff = line.split(' ')
            block = buff[7]
    return block, result

prev = None
last = prev
while True:
    last, result = get_last_block()
    if last != prev:
        print('=' * 80)
        display(result)
        
        block_full = ton.liteClient.Run(f'dumpblock {block}')
        display(block_full.split('\n'))
        
#         shards = ton.GetShards(last)
#         display(shards)
#         for shard in shards:
#             display(ton.GetShards(shard['block']))
    prev = last

In [ ]:
block_ext_id = '(-1,8000000000000000,16136943):8F47FEC07DF718A89FFA3479EC8EA68A0A53EB22F15E258BEF3718C5154D5F73:32F26ACA68A95946ACD0B4BB7ABA75E1ADB232362E8B765BA2EE7EA7CD3BFC95'
tran_list = ton.GetTransactions(block_ext_id)
print(block_ext_id)
display(tran_list)

for idx, tran in enumerate(tran_list):
    chain_id = -1
    tran_info = ton.GetTrans(block_ext_id, f"{chain_id}:{tran['account']}", tran['lt'])
    display(tran['id'], tran_info)

# TPS, validators, shards

In [ ]:
last_block = ton.GetLastBlock()
last_block

In [ ]:
ton.GetTransactions('(-1,8000000000000000,16110296):6AF0B76A2FD2CF15F8FDF08E3BE927D38919DC89C76BB25AC4282EBA37AA7ADD:B5925025E14C70317D4FC41F3697A2C42A74FA93E18C971D41B4AD95EA4AF981')

In [ ]:
ton.GetLastBlock()

In [37]:
validators = ton.GetValidatorsList()

[debug]   17.11.2021, 03:32:40.665 (UTC)  <MainThread>  start GetConfig34 function
[debug]   17.11.2021, 03:32:40.692 (UTC)  <MainThread>  start GetValidatorsLoad function (1637117900, 1637119900)


In [39]:
len(validators)

147

In [40]:
len({x['adnlAddr'] for x in validators})

147

In [ ]:
ton = M.MyTonCore()


data = {}
data['tps'] = ton.GetStatistics('tpsAvg')[0]

# block rate
data['block_rate'] = {
    'masterchain': 3.0,
    'basechain': 2.0,
}

data['shardchains'] = ton.GetShardsNumber()

# validators
validators = ton.GetValidatorsList()
validators_total = len(validators)
validators_online = len(list(filter(lambda x: x['online'], validators)))
data['validators'] = {
    'online': validators_online,
    'total': validators_total
}

data

# dht servers

# check liteservers

In [ ]:
# Считать список серверов
filePath = ton.liteClient.configPath
file = open(filePath, 'rt')
text = file.read()
file.close()
data = json.loads(text)

# Пройтись по серверам
result = list()
liteservers = data.get("liteservers")
for index, server in enumerate(liteservers):
    ip = int2ip(server['ip'])
    port = server['port']
    addr = f"{ip}:{port}"
    
    # working
    working = False
    try:
        ton.liteClient.Run("last", index=index)
        working = True
    except: pass
    
    # ping
    if working:
        runs = 10
        resp_time = measure_latency(ip, port=port, runs=runs, wait=0)
        ping = sum(resp_time) / runs
    else:
        ping = None
    
    # status
    try:
        resp = ton.liteClient.Run("last", index=index)
        if 'sync' in resp:
            print(resp)
    except: pass
    
    loc = {
        'addr': addr,
        'working': working,
        'response_time': ping,
        'syncronized': 'unknown',
    }
    print(loc)
    break
    result.append(loc)
#end for

# Записать данные в базу
result

# Services uptime

In [ ]:
services_list = [{'name': 'ton.sh',
                  'api': 'https://api.ton.sh/getAddressInformation?address=EQCD39VS5jcptHL8vMjEXrzGaRcCVYto7HUn4bpAOg8xqB2N',
                  'ip': 'https://ton.sh'},
                 {'name': 'toncenter.com',
                  'api': 'https://toncenter.com/api/v2/getAddressInformation?address=0QCyt4ltzak71h6XkyK4ePfZCzJQDSVUNuvZ3VE7hP_Q-GTE',
                  'ip': 'https://toncenter.com'}]

services = []
for service in services_list:
    name = service['name']
    url = service['api']
    ip = server['ip']
    available = R.get(url, timeout=2).status_code == 200
    loc = {'name': name, 'available': available}
    services.append(loc)
services

# Governance

In [ ]:
config17 = ton.GetConfig17()
config15 = ton.GetConfig15()
config34 = ton.GetConfig34()
config36 = ton.GetConfig36()

fullElectorAddr = ton.GetFullElectorAddr()


# get times
startWorkTime = ton.GetActiveElectionId(fullElectorAddr)
election_status = "open"
if startWorkTime == 0:
    startWorkTime = config36.get("startWorkTime")
    if startWorkTime is None:
        startWorkTime = config34.get("startWorkTime")
    election_status = "closed" 

electionsStartBefore = config15["electionsStartBefore"]
electionsEndBefore = config15["electionsEndBefore"]
validatorsElectedFor = config15["validatorsElectedFor"]

# compute election times
startElection = startWorkTime - electionsStartBefore
endElection = startWorkTime - electionsEndBefore
startNextElection = startElection + validatorsElectedFor

startElection = datetime.fromtimestamp(startElection)
endElection = datetime.fromtimestamp(endElection)
startNextElection = datetime.fromtimestamp(startNextElection)

elections = {
    'status': election_status,
    'start': startElection,
    'end': endElection,
    'start_next': startNextElection
}
elections

In [ ]:
offers = ton.GetOffersNumber()
offers_list = ton.GetOffers()
offers['offers_list'] = offers_list

offers

In [ ]:
complaints = ton.GetComplaintsNumber()
complaints_list = ton.GetComplaints()
complaints['complaints_list'] = complaints_list

complaints

In [ ]:
governance = {
    'elections': elections,
    'offers': offers,
    'complaints': complaints
}
governance

# Bridge check

In [ ]:
from bs4 import BeautifulSoup


bridge_list = [{'name': 'ton-eth','url': 'https://ton.org/bridge/', 
                'address': 'Ef_dJMSh8riPi3BTUTtcxsWjG8RLKnLctNjAM4rw8NN-xWdr'},
               {'name': 'ton-bsc', 'url': 'https://ton.org/bridge/bsc', 
                'address': 'Ef9NXAIQs12t2qIZ-sRZ26D977H65Ol6DQeXc5_gUNaUys5r'},
               {'name': 'ton-other', 'url': 'https://ton.org/bridge/exmo', 
                'address': ''}]

bridges = []
for bridge_dict in bridge_list:
    resp = R.get(bridge_dict['url'])
    
    # web page status
    web_page_status = 'not available'
    if resp.status_code == 200:
        page = BeautifulSoup(resp.content, features="html.parser")
        button = page.find_all(attrs={'id': ['transferButton']})
        if len(button) > 0:
            web_page_status = 'available'
            
    # api status
    smart_contract_state = None
    if bridge_dict['address']:
        cmd = 'runmethod {address} get_bridge_data'.format(**bridge_dict)
        resp = ton.liteClient.Run(cmd)
        for row in resp.split('\n'):
            if 'result:' in row and '[' in row:
                row = row[row.find('[')+1:row.find(']')].strip()
                row = row.split()
                if len(row) > 0:
                    smart_contract_state = int(row[0])
    bridges.append({
        'name': bridge_dict['name'],
        'url': bridge_dict['url'],
        'web_page_status': web_page_status,
        'smart_contract_state': smart_contract_state
    })
bridges

# Backup

# OLD

In [ ]:
lite = M.LiteClient()

In [ ]:
ton.GetConfig12()

In [ ]:
ton.GetConfig15()

In [ ]:
config36 = ton.GetConfig36()
config36

In [ ]:
shardsNumber = ton.GetShardsNumber()
print('shards_number:', shardsNumber)

In [ ]:
online_validators = ton.GetOnlineValidators()
validator_efficiency = ton.GetValidatorEfficiency()

In [ ]:
config15 = ton.GetConfig15()
config17 = ton.GetConfig17()

In [ ]:
statistics = ton.GetSettings("statistics")
statistics

In [ ]:
rootWorkchainEnabledTime_int = ton.GetRootWorkchainEnabledTime()
rootWorkchainEnabledTime_int

In [ ]:
validators_info = ton.GetValidatorsLoad()
validators_info[0]

In [ ]:
compliants_info = ton.GetComplaints()
compliants_info

In [ ]:
ton.GetAdnlAddr()

In [ ]:
statistics = ton.GetSettings("statistics")
tpsAvg = ton.GetStatistics("tpsAvg", statistics)
netLoadAvg = ton.GetStatistics("netLoadAvg", statistics)
disksLoadAvg = ton.GetStatistics("disksLoadAvg", statistics)
disksLoadPercentAvg = ton.GetStatistics("disksLoadPercentAvg", statistics)

statistics

In [ ]:
def PrintStatus(args):
    opt = None
    if len(args) == 1:
        opt = args[0]
    rootWorkchainEnabledTime_int = ton.GetRootWorkchainEnabledTime()
    config34 = ton.GetConfig34()
    config36 = ton.GetConfig36()
    totalValidators = config34["totalValidators"]
    onlineValidators = None
    validatorEfficiency = None
    if opt != "fast":
        onlineValidators = ton.GetOnlineValidators()
        validatorEfficiency = ton.GetValidatorEfficiency()
    if onlineValidators:
        onlineValidators = len(onlineValidators)
    oldStartWorkTime = config36.get("startWorkTime")
    if oldStartWorkTime is None:
        oldStartWorkTime = config34.get("startWorkTime")
    shardsNumber = ton.GetShardsNumber()
    validatorStatus = ton.GetValidatorStatus()
    config15 = ton.GetConfig15()
    config17 = ton.GetConfig17()
    fullConfigAddr = ton.GetFullConfigAddr()
    fullElectorAddr = ton.GetFullElectorAddr()
    startWorkTime = ton.GetActiveElectionId(fullElectorAddr)
    validatorIndex = ton.GetValidatorIndex()
    validatorWallet = ton.GetLocalWallet(ton.validatorWalletName)
    dbSize = ton.GetDbSize()
    offersNumber = ton.GetOffersNumber()
    complaintsNumber = ton.GetComplaintsNumber()
    statistics = ton.GetSettings("statistics")
    tpsAvg = ton.GetStatistics("tpsAvg", statistics)
    netLoadAvg = ton.GetStatistics("netLoadAvg", statistics)
    disksLoadAvg = ton.GetStatistics("disksLoadAvg", statistics)
    disksLoadPercentAvg = ton.GetStatistics("disksLoadPercentAvg", statistics)
    if validatorWallet is not None:
        validatorAccount = ton.GetAccount(validatorWallet.addr)
    else:
        validatorAccount = None
    PrintTonStatus(startWorkTime, totalValidators, onlineValidators, shardsNumber, offersNumber, complaintsNumber, tpsAvg)
    PrintLocalStatus(validatorIndex, validatorEfficiency, validatorWallet, validatorAccount, validatorStatus, dbSize, netLoadAvg, disksLoadAvg, disksLoadPercentAvg)
    PrintTonConfig(fullConfigAddr, fullElectorAddr, config15, config17)
    PrintTimes(rootWorkchainEnabledTime_int, startWorkTime, oldStartWorkTime, config15)